<a href="https://colab.research.google.com/github/power80203/AtoZ/blob/master/ocproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

引入套件 跟 確認GPU 是否正常啟用

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import keras.optimizers as optimizers
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Read data from google drive 

In [7]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


# from google.colab import files
# uploaded = files.upload()

# import io
# df2 = pd.read_csv(io.BytesIO(uploaded['standed_eight_hour_merge.csv']))
# # Dataset is now stored in a Pandas Dataframe


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:

!ls "/content/drive/My Drive/ocdata"
df2 = pd.read_csv('/content/drive/My Drive/ocdata/standed_eight_hour_merge.csv')

standed_eight_hour_merge.csv


In [0]:

def judge_fail(i):
    """
    check each y and change to target label

    """
    if i < 1 :
        return 1
    else:
        return 0



X = df2.iloc[:,1:426].values
y = df2.iloc[:,428].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


print("X 降維前訓練資料的維度為  ", X_train.shape)
print("X 降維前測試資料的維度為  ",X_test.shape)
print("y 訓練資料的維度為  ",y_train.shape)
print("y 測試資料的維度為  ",y_test.shape)


from sklearn.decomposition import PCA
pca = PCA(n_components = 80)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance = pca.explained_variance_ratio_


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print("X 降維後訓練資料的維度為  ", X_train.shape)
print("X 降維後測試資料的維度為  ",X_test.shape)
print("y 訓練資料的維度為  ",y_train.shape)
print("y 測試資料的維度為  ",y_test.shape)



X 降維前訓練資料的維度為   (11674, 425)
X 降維前測試資料的維度為   (2919, 425)
y 訓練資料的維度為   (11674,)
y 測試資料的維度為   (2919,)
X 降維後訓練資料的維度為   (11674, 80, 1)
X 降維後測試資料的維度為   (2919, 80, 1)
y 訓練資料的維度為   (11674,)
y 測試資料的維度為   (2919,)


In [35]:
print(" ")
print("開始進行建模")

print(" ")

from keras.layers import Dense, Dropout
from keras.layers import Embedding

model = Sequential()
model.add(LSTM(256, return_sequences= True,activation = 'tanh',
               input_shape=(None , 1 )))  # returns a sequence of vectors of dimension 256
           
model.add(LSTM(256, return_sequences= True, activation = 'tanh'))  # returns a sequence of vectors of dimension 128


model.add(LSTM(256,activation = 'tanh'))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False,clipnorm=1.)

print("建模完成, 開始進行訓練")
model.compile(loss='binary_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])


#########################################################
# set LSTM model

print( )
print("訓練完成 開始進行預測")

model.fit(X_train, y_train, batch_size=128, epochs=100)

y_pred = model.predict(X_test)


y_pred = (y_pred > 0.5)

#print(confusion_matrix(y_pred,y_test))

print("模型準確率 為 ",accuracy_score(y_test,y_pred))


 
開始進行建模
 
建模完成, 開始進行訓練

訓練完成 開始進行預測
Epoch 1/100
11674/11674 [==============================] - 86s 7ms/step - loss: 0.5696 - acc: 0.7447
Epoch 2/100
11674/11674 [==============================] - 79s 7ms/step - loss: 0.5613 - acc: 0.7500
Epoch 3/100
11674/11674 [==============================] - 78s 7ms/step - loss: 0.5566 - acc: 0.7487
Epoch 4/100
11674/11674 [==============================] - 79s 7ms/step - loss: 0.5540 - acc: 0.7500
Epoch 5/100
11674/11674 [==============================] - 78s 7ms/step - loss: 0.5469 - acc: 0.7488
Epoch 6/100
11674/11674 [==============================] - 78s 7ms/step - loss: 0.5348 - acc: 0.7492
Epoch 7/100
11674/11674 [==============================] - 76s 7ms/step - loss: 0.5024 - acc: 0.7497
Epoch 8/100
11674/11674 [==============================] - 76s 6ms/step - loss: 0.4886 - acc: 0.7640
Epoch 9/100
11674/11674 [==============================] - 76s 6ms/step - loss: 0.4649 - acc: 0.7715
Epoch 10/100
11674/11674 [============================

模型準確率 為 0.7334703665638918
